In [36]:
# A script to calculate tolerance factors of ABX3 perovskites using bond valences from 2016
# Data from the International Union of Crystallography
# Author: Nick Wagner
import pandas as pd
import numpy as np

In [37]:
bv = pd.read_csv("../Bond_valences2016.csv")
bv.head()

,Atom1,Atom1_valence,Atom2,Atom2_valence,Ro,B,ref_id
0,Ac,3,O,-2,2.240,0.370,b
1,Ac,3,F,-1,2.130,0.370,b
2,Ac,3,Cl,-1,2.630,0.370,b
3,Ac,3,Br,-1,2.750,0.400,p
4,Ag,1,O,-2,1.875,0.359,bs


In [41]:
def calc_tol_factor(ion_list, valence_list):
    if len(ion_list) != 3:
        print("WARNING: Is this an ABX3 perovskite?")
    for i in range(len(valence_list)):
        if valence_list[i][-1] == '-':
            valence_list[i] = valence_list[i][-1] + valence_list[i][:-1]
    for i in range(len(valence_list)):
        valence_list[i] = int(valence_list[i].strip("+"))

    
    AO_row = bv[(bv['Atom1'] == ion_list[0]) & (bv['Atom1_valence'] == valence_list[0])
                  & (bv['Atom2'] == ion_list[2]) & (bv['Atom2_valence'] == valence_list[2])]
    BO_row = bv[(bv['Atom1'] == ion_list[1]) & (bv['Atom1_valence'] == valence_list[1])
                  & (bv['Atom2'] == ion_list[2]) & (bv['Atom2_valence'] == valence_list[2])]
    avg_AO_row = AO_row.mean(axis=0) # If multiple values exist, take average
    avg_BO_row = BO_row.mean(axis=0)
    
    AO_bv = avg_AO_row['Ro']-avg_AO_row['B'] * np.log(avg_AO_row['Atom1_valence']/12)
    BO_bv = avg_BO_row['Ro']-avg_BO_row['B'] * np.log(avg_BO_row['Atom1_valence']/6)               
      
    tol_fact = AO_bv / (2**0.5 * BO_bv)
    
    return tol_fact


In [46]:
# Test using BaMnO3
print(calc_tol_factor(['Ba', 'Mn','O'], ['2+', '4+', '2-']))

1.09738895723
